# importing all the packages

In [24]:
import numpy as np
import pandas as pd
import requests
 

### loading the page, using pandas package to find the form and removing the 'Not assigned' lines in 'Borough'

In [29]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
web_con=requests.get(url) 
web_con.encoding='utf-8'
#soup=bs(web_con.text,'lxml')
form1=pd.read_html(url)[0]   # There are 3 data frames in the page, [0] happens to be the one we want
print(form1)
test=form1['Borough'].isin(['Not assigned'])
~test
form=form1[~test] 
form.head(7)

    Postcode           Borough           Neighborhood
0        M1A      Not assigned           Not assigned
1        M2A      Not assigned           Not assigned
2        M3A        North York              Parkwoods
3        M4A        North York       Victoria Village
4        M5A  Downtown Toronto           Harbourfront
..       ...               ...                    ...
282      M8Z         Etobicoke              Mimico NW
283      M8Z         Etobicoke     The Queensway West
284      M8Z         Etobicoke  Royal York South West
285      M8Z         Etobicoke         South of Bloor
286      M9Z      Not assigned           Not assigned

[287 rows x 3 columns]


,Postcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Queen's Park,Not assigned
9,M9A,Downtown Toronto,Queen's Park


### Replacing the 'Not assigned' in 'Neighborhood' with the value in 'Borough'

In [30]:
form.Neighborhood[form.Neighborhood=='Not assigned']=form.Borough
form.head(7)

,Postcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Queen's Park,Queen's Park
9,M9A,Downtown Toronto,Queen's Park


### Getting the 'Postcode' data series, for the future building the data frame
##### Note it was extracted as 'keys' from the dictionaries.  The product of 'groupby' command cannot be directly converted to dictionary, has to be through list

In [33]:
grouped=form.groupby('Postcode')

post_1=dict(list(grouped)).keys() 
post= pd.Series(list(post_1))
post

0      M1B
1      M1C
2      M1E
3      M1G
4      M1H
      ... 
98     M9N
99     M9P
100    M9R
101    M9V
102    M9W
Length: 103, dtype: object

### Getting the 'Neighborhood' data series, for the future building the data frame
#####  Note that the 'join' step was operated on the dataframes made by the 'groupby'  

In [13]:
Neigh_1= grouped.apply(lambda x: ','.join(x['Neighborhood']))
Neigh= pd.Series(list(Neigh_1))
Neigh

0                                          Rouge,Malvern
1                   Highland Creek,Rouge Hill,Port Union
2                        Guildwood,Morningside,West Hill
3                                                 Woburn
4                                              Cedarbrae
                             ...                        
98                                                Weston
99                                             Westmount
100    Kingsview Village,Martin Grove Gardens,Richvie...
101    Albion Gardens,Beaumond Heights,Humbergate,Jam...
102                                            Northwest
Length: 103, dtype: object

### Getting the 'Borough' data series, for the future building the data frame
#####  Because each postcode is supposed to has only one Borough, so the 'unique' operation can be done on data frame ['Borough'] column

In [20]:
Bor_1=grouped.apply(lambda x: x['Borough'].unique())
Bor=pd.Series(list(Bor_1))
Bor

0      [Scarborough]
1      [Scarborough]
2      [Scarborough]
3      [Scarborough]
4      [Scarborough]
           ...      
98            [York]
99       [Etobicoke]
100      [Etobicoke]
101      [Etobicoke]
102      [Etobicoke]
Length: 103, dtype: object

## Building the data frame

In [22]:
form_new=pd.DataFrame({'PostCode': post, 'Borough': Bor, 'Neighborhood':Neigh})
form_new.head()

,PostCode,Borough,Neighborhood
0,M1B,[Scarborough],"Rouge,Malvern"
1,M1C,[Scarborough],"Highland Creek,Rouge Hill,Port Union"
2,M1E,[Scarborough],"Guildwood,Morningside,West Hill"
3,M1G,[Scarborough],Woburn
4,M1H,[Scarborough],Cedarbrae


# The shape

In [36]:
form_new.shape

(103, 3)